In [1]:
import bokeh.plotting as bk
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.append('../') # go to parent dir
import numpy as np
np.set_printoptions(precision=3, formatter={'float': '{: 0.3f}'.format})
np.seterr('warn')
import os
import cProfile, pstats, io

In [3]:
from bokeh.palettes import Category20
colors = Category20[20]

### Load Data

In [4]:
path = os.path.join(os.getcwd(), 'data')
frame_start, frame_end = -1, 100
measurements = np.load(os.path.join(path, 'simulated_data' + '.npy'))
gt_bboxes = np.load(os.path.join(path, 'gt_bboxes' + '.npy'))
if frame_end is -1:
    measurements = measurements[measurements['ts'] >= frame_start]
    gt_bboxes = gt_bboxes[gt_bboxes['ts'] >= frame_start]
else:
    measurements = measurements[ (measurements['ts'] >= frame_start) & (measurements['ts'] <= frame_end)]
    gt_bboxes = gt_bboxes[ (gt_bboxes['ts'] >= frame_start) & (gt_bboxes['ts'] <= frame_end)]

## Star Convex Gaussian Process Tracker

### Create Tracker

In [5]:
steps = max(measurements['ts']) + 1

# tracker config
dt = 0.04
sa_sq = 1.5 ** 2
somega_sq = 0.1 ** 2
q = np.zeros((5, 5), dtype='f4')
q[2, 2] = somega_sq * dt ** 3 / 3.0
q[2, 4] = somega_sq * dt ** 2 / 2.0
q[4, 2] = q[2, 4]
q[4, 4] = somega_sq * dt
q[3, 3] = sa_sq * dt
# q[0, 0] = 1e-3 * dt
# q[1, 1] = 1e-3 * dt

uf = np.linspace(0, 2 * np.pi, 25, endpoint=False)
xf = 0.8 * np.ones(len(uf), dtype='f8')

config = {
    'steps': steps + 1,
    'd': 2,
    'sd': 5,
    'alpha': 1.0,
    'beta': 2.0,
    'kappa': 2.0,
    'sa_sq': sa_sq,
    'somega_sq': somega_sq,
    'q': q,
    'r': 0.1 ** 2 * np.identity(2),
    #          x    y phi vx o sx sy
    'init_m': np.asarray([6.5, 2.5, 0.00, 12, 0]),
    'sl': np.pi / 8.0,
    'sf': 0.2,
    'sr': 0.1,
    'alpha_f': 0.01,
    'uf': uf,
    'xf': xf,
    'xf0': xf,
    'init_c': 2 ** 2 * q + np.diag([0.25, 0.25, 0, 0, 0]),
    'scale_correction': True,
    'orientation_correction': True,
    'fixed_size': False,
}

from models import ScGpTracker as Tracker

### Run Tracker

In [6]:
# tracker definition
tracker = Tracker(dt=dt, **config)

pr = cProfile.Profile()
for i in range(steps):
    scan = measurements[measurements['ts'] == i]
    pr.enable()
    tracker.step(scan)
    pr.disable()

estimates, log_lik = tracker.extract()
bboxes = tracker.extrackt_bbox()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
ps.print_stats(10)

print('total time: {:f}, total steps: {:d}, average step time: {:f}'.format(
    ps.total_tt, max(measurements['ts']) - 2, ps.total_tt / (max(measurements['ts']) - 2)))
print(s.getvalue())

total time: 1.869246, total steps: 97, average step time: 0.019271
         109730 function calls in 1.869 seconds

   Ordered by: cumulative time
   List reduced from 103 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.006    0.000    1.869    0.019 ../tracker.py:47(step)
      100    0.100    0.001    1.788    0.018 ../models/scgp.py:207(correct)
      100    1.170    0.012    1.176    0.012 /Users/Jens/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:464(inv)
     1200    0.212    0.000    0.212    0.000 {built-in method numpy.core.multiarray.dot}
      100    0.093    0.001    0.208    0.002 /Users/Jens/anaconda3/lib/python3.6/site-packages/scipy/linalg/special_matrices.py:475(block_diag)
      100    0.028    0.000    0.076    0.001 ../models/scgp.py:172(predict)
      200    0.007    0.000    0.069    0.000 /Users/Jens/anaconda3/lib/python3.6/site-packages/numpy/core/einsumfunc.py:819(einsum)
      

### Calculate Wasserstein Metric

In [7]:
from metric import point_set_wasserstein_distance as pt_wsd
from misc import convert_rectangle_to_eight_point
eight_pts = convert_rectangle_to_eight_point(bboxes[1:])  # drop prior bounding box
eight_pts_gt = convert_rectangle_to_eight_point(gt_bboxes)

wsd = np.zeros(len(gt_bboxes), dtype='f8')
for i, (pts, gt_pts) in enumerate(zip(eight_pts, eight_pts_gt)):
    wsd[i] = pt_wsd(pts, gt_pts, p=2.0)

### data source version

In [8]:
from bokeh.models import ColumnDataSource

stride = 5

est_source = ColumnDataSource(
        data={
            'ts': estimates['ts'],
            'm_x': estimates['m'][:, 0],
            'm_y': estimates['m'][:, 1],
            'phi': estimates['m'][:, 2],
            'v': estimates['m'][:, 3],
            'uf': estimates['uf'],
            'xf': estimates['xf'],
            'loglik': log_lik,
        }
    )

meas_source = ColumnDataSource(
        data={
            'ts': measurements['ts'],
            'x': measurements['xy'][:, 0],
            'y': measurements['xy'][:, 1],
        }
)

sel = measurements['ts'] % stride == 0
meas_source_sel = ColumnDataSource(
        data={
            'ts': measurements['ts'][sel],
            'x': measurements['xy'][sel, 0],
            'y': measurements['xy'][sel, 1],
        }
)

sel = bboxes['ts'] % stride == 0
bbox_source = ColumnDataSource(
        data={
            'ts': bboxes['ts'][sel],
            'm_x': bboxes['center_xy'][sel, 0],
            'm_y': bboxes['center_xy'][sel, 1],
            'phi': bboxes['orientation'][sel],
            'l': bboxes['dimension'][sel, 0],
            'w': bboxes['dimension'][sel, 1],
        }
)

gt_bbox_source = ColumnDataSource(
        data={
            'ts': gt_bboxes['ts'],
            'm_x': gt_bboxes['center_xy'][:, 0],
            'm_y': gt_bboxes['center_xy'][:, 1],
            'phi': gt_bboxes['orientation'],
            'l': gt_bboxes['dimension'][:, 0],
            'w': gt_bboxes['dimension'][:, 1],
            'wsd': wsd,
        }
)

sel = gt_bboxes['ts'] % stride == 0
gt_bbox_source_sel = ColumnDataSource(
        data={
            'ts': gt_bboxes['ts'][sel],
            'm_x': gt_bboxes['center_xy'][sel, 0],
            'm_y': gt_bboxes['center_xy'][sel, 1],
            'phi': gt_bboxes['orientation'][sel],
            'l': gt_bboxes['dimension'][sel, 0],
            'w': gt_bboxes['dimension'][sel, 1],
        }
)

In [9]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, BoxSelectTool, PanTool, BoxZoomTool, WheelZoomTool, UndoTool, RedoTool, ResetTool, SaveTool

TOOLS = [PanTool(), BoxSelectTool(), BoxZoomTool(), WheelZoomTool(), UndoTool(), RedoTool(), ResetTool(), SaveTool()]

top = figure(tools=TOOLS, width=800, height=350, title=None, match_aspect=True, output_backend="webgl")
meas_scatter = top.circle('x', 'y', color='#303030', legend='measurements', size=1, alpha=0.2, source=meas_source)
track_plot = top.line('m_x', 'm_y', legend='track', source=est_source)
hover = HoverTool(renderers=[track_plot],
        tooltips=[
            ("index", "$index"),
            (r"(x, y, phi, v)", "(@m_x, @m_y, @phi, @v)"),
            ("(length, width)", "(@d_x, @d_y)"),
        ]
    )
top.add_tools(hover)

bottom = figure(tools=TOOLS, width=800, height=350, title=None, 
                x_range=top.x_range, y_range=top.y_range, output_backend="webgl")
bottom.circle('x', 'y', color='#303030', legend='measurements', size=1, alpha=0.2, source=meas_source_sel)
bottom.rect(x='m_x', y='m_y', width='l', height='w', angle='phi', color="#CAB2D6", 
            fill_alpha=0.2, source=bbox_source, legend='bounding boxes')
bottom.rect(x='m_x', y='m_y', width='l', height='w', angle='phi', color='#98df8a', 
            fill_alpha=0.2, source=gt_bbox_source_sel, legend='ground truth boxes')

sel = estimates['ts'] % stride == 0
poly_pts_x = [np.cos(est['uf'] + est['m'][2]) * est['xf'] + est['m'][0] for est in estimates[sel]]
poly_pts_y = [np.sin(est['uf'] + est['m'][2]) * est['xf'] + est['m'][1] for est in estimates[sel]]
bottom.patches(poly_pts_x, poly_pts_y, color=colors[0], fill_alpha=0.2, legend='shape')

bottom.legend.click_policy="hide"

p = gridplot([[top], [bottom]])
show(p)

### Statistics

In [10]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.layouts import gridplot, column

f_wsd = figure(tools=TOOLS, width=800, height=350, title='Wasserstein Distance')
f_log_lik = figure(tools=TOOLS, width=800, height=350, title='Likelihood', x_range=f_wsd.x_range)
f_wsd.line('ts', 'wsd', legend='wasserstein distance', source=gt_bbox_source, color=colors[0])
f_log_lik.line('ts', 'loglik', legend='log likelihood', source=est_source, color=colors[0])
pan1 = Panel(child=column([f_wsd, f_log_lik]), title="Likelihood and Wasserstein Distance")

f_position = figure(tools=TOOLS, width=800, height=350, title='Position')
f_orientation = figure(tools=TOOLS, width=800, height=350, title='Orientation', x_range=f_position.x_range)
f_velocity = figure(tools=TOOLS, width=800, height=350, title='Velocity', x_range=f_position.x_range)
f_dimension = figure(tools=TOOLS, width=800, height=350, title='Dimension', x_range=f_position.x_range)
f_position.line('ts', 'm_x', legend='x position', source=est_source, color=colors[0])
f_position.line('ts', 'm_y', legend='y position', source=est_source, color=colors[2])
f_position.line('ts', 'm_x', legend='ground truth x position', source=gt_bbox_source, color=colors[1])
f_position.line('ts', 'm_y', legend='ground truth y position', source=gt_bbox_source, color=colors[3])
f_orientation.line('ts', 'phi', legend='orientation', source=est_source, color=colors[0])
f_orientation.line('ts', 'phi', legend='ground truth orientation', source=gt_bbox_source, color=colors[1])
f_velocity.line('ts', 'v', legend='velocity', source=est_source, color=colors[0])
# f_velocity.line('ts', 'v', legend='ground truth orientation', source=gt_bbox_source, color=colors[3])
f_dimension.line('ts', 'l', legend='x dimension scaling', source=bbox_source, color=colors[0])
f_dimension.line('ts', 'w', legend='y dimension scaling', source=bbox_source, color=colors[2])
f_dimension.line('ts', 'l', legend='ground truth x dimension', source=gt_bbox_source, color=colors[1])
f_dimension.line('ts', 'w', legend='ground truth y dimension', source=gt_bbox_source, color=colors[3])
pan2 = Panel(child=column([f_position, f_orientation, f_velocity, f_dimension]), title="Kinematic and Extent")

for f in [f_wsd, f_log_lik, f_position, f_orientation, f_velocity, f_dimension]:
    f.legend.click_policy="mute"

tabs = Tabs(tabs=[pan1, pan2])
show(tabs)